# Imports

In [31]:
import pandas as pd

# Load data

In [32]:
df = pd.read_csv('data/raw/creditcard.csv')
df.head(3)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0


# Split

In [33]:
quantile_30 = df['Time'].quantile(0.03)

df_train = df.loc[df['Time'] < quantile_30]
len(df_train) / len(df)

0.03000277380822803

In [34]:
df_train['Class'].value_counts()

0    8517
1      28
Name: Class, dtype: int64

In [35]:
df_test_positive = df.loc[(df['Time'] >= quantile_30)
                         & (df['Class'] == 1)]
display(
    df_test_positive['Class'].value_counts()
    , df_test_positive.head()
)

1    464
Name: Class, dtype: int64

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
8615,11629.0,-3.891192,7.098916,-11.426467,8.607557,-2.065706,-2.985288,-8.138589,2.973928,-6.272790,...,1.757085,-0.189709,-0.508629,-1.189308,1.188536,0.605242,1.881529,0.875260,1.00,1
8617,11635.0,0.919137,4.199633,-7.535607,7.426940,1.118215,-2.886722,-1.341036,0.363933,-2.203224,...,0.316094,0.055179,0.210692,-0.417918,-0.911188,0.466524,0.627393,0.157851,1.00,1
8842,12093.0,-4.696795,2.693867,-4.475133,5.467685,-1.556758,-1.549420,-4.104215,0.553934,-1.498468,...,0.573898,-0.080163,0.318408,-0.245862,0.338238,0.032271,-1.508458,0.608075,0.00,1
8845,12095.0,-4.727713,3.044469,-5.598354,5.928191,-2.190770,-1.529323,-4.487422,0.916392,-1.307010,...,0.650988,0.254983,0.628843,-0.238128,-0.671332,-0.033590,-1.331777,0.705698,30.39,1
8972,12393.0,-4.064005,3.100935,-1.188498,3.264633,-1.903562,0.320351,-0.954940,-3.277535,2.820829,...,1.688665,-0.078845,0.193731,0.479496,-0.506603,-0.409863,-3.036271,-0.630605,179.66,1


In [36]:
df_test_negative = df.loc[(df['Time'] >= quantile_30)
                         & (df['Class'] == 0)]

df_test_negative_sample = df_test_negative.sample(frac=0.01, random_state=16)
df_test_negative_sample.shape

(2758, 31)

# Concat new test

In [37]:
df_test = pd.concat([df_test_positive, df_test_negative_sample])
df_test['Class'].value_counts()

0    2758
1     464
Name: Class, dtype: int64

# Drop fraud from train

In [38]:
print(df_train.shape)
df_train = df_train.loc[df_train['Class'] == 0]
print(df_train.shape)

(8545, 31)
(8517, 31)


In [39]:
df_train.drop(columns=['Class'], inplace=True)

# Save df

In [40]:
df_train.to_csv('data/ready/novelty_detection/train.csv', index=False)

In [41]:
df_test.to_csv('data/ready/novelty_detection/test.csv', index=False)